In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#TopUniversities.com
r = requests.get("https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508602165358")
dic = r.json()['data']
dic[0]

In [ ]:
for univ in dic[:200]:
    #Get information from second page
    r2 = requests.get("https://www.topuniversities.com" + univ['url'])
    univ['soup'] = BeautifulSoup(r2.text, 'html.parser')

In [ ]:
data_list = []
for univ in dic[:200]:
    #First page information
    row = {'Name':univ['title'],'Rank':univ['rank_display'],'Country':univ['country'],'Region':univ['region']}
    
    #Second page information
    try:
        row['Number of students'] = univ['soup'].find('div',class_ = "total student").find('div',class_="number").string[1:-1]
        row['Number of international students'] = univ['soup'].find('div',class_ = "total inter").find('div',class_="number").string[1:-1]
    
        row['Number of faculty members'] = univ['soup'].find('div',class_ = "total faculty").find('div',class_="number").string[1:-1]
        row['Number of international members'] = univ['soup'].find('div',class_ = "inter faculty").find('div',class_="number").string[1:-1]
    except:
        print(univ['title'], univ['rank_display'])
        
    data_list.append(row)

In [ ]:
data = pd.DataFrame(data_list)
data = data[['Name','Rank','Country','Region','Number of faculty members','Number of international members','Number of students', 'Number of international students']]#'Number of faculty members','Number of international members']]

pd.set_option('display.max_rows', 200)
data

#199-201 wrong -> should be 198-200

In [ ]:
##TimesHigherEducation

r3 = requests.get("https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json")
list_univ = r3.json()['data']

In [ ]:
data_2 = pd.DataFrame(list_univ[:200])
data_2 = data_2[['name','location','rank','stats_number_students','stats_pc_intl_students','stats_student_staff_ratio']]
data_2.columns = ['Name','Country','Rank','Number of students','Percentage of international students','Students-Staff ratio']

data_2

In [ ]:
#Wikibooks
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]/max(len(s1),len(s2))

In [ ]:
data['dummy'] = 1
data_2['dummy'] = 1
#Compressing 'University' and 'Institute'
data['Name'] = data['Name'].apply(lambda x: x.replace('University','%').replace('Institute','#'))
data_2['Name'] = data_2['Name'].apply(lambda x: x.replace('University','%').replace('Institute','#'))

merge_data = pd.merge(data,data_2,on=['dummy'],suffixes=['_1','_2'])
merge_data['dist'] = merge_data.apply(lambda x: levenshtein(x['Name_1'],x['Name_2']),axis = 1)

In [ ]:
data_tronc = merge_data[merge_data['dist'] < 0.4][['Name_1','Name_2','dist']]
data_tronc

In [ ]:
from munkres import Munkres
m = Munkres()
names_1 = list(set(data_tronc['Name_1']))
names_2 = list(set(data_tronc['Name_2']))
index_1 = {val:i for i,val in enumerate(names_1)}
index_2 = {val:i for i,val in enumerate(names_2)}
matrix = [[1]*(len(names_1) + len(names_2)) for x in [0]*(len(names_1) + len(names_2))]

for ind,x in data_tronc.iterrows():
    matrix[index_1[x['Name_1']]][index_2[x['Name_2']]] = x['dist']
indexes = m.compute(matrix)
    
pair = [[names_1[i],names_2[j]] for i,j in indexes if i < len(names_1) and j < len(names_2) and matrix[i][j] < 0.4]

match_pair = pd.DataFrame(pair)
match_pair.columns = ['Name_1','Name_2']
match_pair

In [ ]:
data.columns = ['Name_1'] + list(data.columns)[1:]
data_2.columns = ['Name_2'] + list(data_2.columns)[1:]

data_left = pd.merge(data,match_pair,how='outer')
data_left

data_right = pd.merge(data_2,match_pair,how='outer')

data_combined = pd.merge(data_left,data_right,how='outer',on=['Name_1','Name_2'],suffixes=['_1','_2'])
data_combined = data_combined[['Name_1','Rank_1','Rank_2','Country_1','Region','Number of students_1','Number of students_2','Number of faculty members','Students-Staff ratio','Number of international members','Percentage of international students']]
data_combined